In [474]:
import random
import pandas as pd
import numpy as np

In [475]:
# inputs
num_tipos_pacientes = 10
num_recursos = 10
num_areas_pop = 30
num_hospitais = 15
size_square = 100

horizonte_planejado = 10

# valores maximos
estadia_max = 5 #round(horizonte_planejado/2)
qtd_pacientes_por_tipo_area_max = 400
qtd_recursos_por_hospital_max = 140
qtd_recursos_por_hospital_min = 100
distancia_max = 50
data_saida_pac_init_max = estadia_max - 1
qtd_pacientes_liberados_init_max = qtd_recursos_por_hospital_max/2


In [476]:
# tipo de paciente vs recursos usados

tipo_paciente = ["TP" + str(i) for i in range(1, num_tipos_pacientes + 1)]
recursos =  ["R" + str(i) for i in range(1, num_recursos + 1)]
usa_recurso = np.random.choice(['0', '1'], size=(num_tipos_pacientes*num_recursos))

tipo_paciente_df_in = np.tile(tipo_paciente, num_recursos)
tipo_paciente_df = sorted(tipo_paciente_df_in)
recursos_df = np.tile(recursos, num_tipos_pacientes)

#print(tipo_paciente)
#print(recursos)
#print(usa_recurso)

paciente_vs_recurso = pd.DataFrame({'Tipo_de_paciente': tipo_paciente_df, 'Recurso': recursos_df, 'Usa_recurso': usa_recurso})
paciente_vs_recurso['Usa_recurso'] = pd.to_numeric(paciente_vs_recurso['Usa_recurso'])
  
#display(paciente_vs_recurso.sort_values(by=['Tipo_de_paciente', 'Recurso']))

#fazer check se para cada tipo de paciente tem pelo menos 1 recurso sendo alocado
for tipo_paciente_in in paciente_vs_recurso['Tipo_de_paciente'].unique():
  filtro_tipo = paciente_vs_recurso['Tipo_de_paciente'] == tipo_paciente_in
 
  soma_recursos = paciente_vs_recurso.loc[filtro_tipo, 'Usa_recurso'].sum()
  
  if soma_recursos == 0:

      filtro_recurso_1 = paciente_vs_recurso['Recurso'] == 'R1'
      
      paciente_vs_recurso.loc[filtro_tipo & filtro_recurso_1, 'Usa_recurso'] = 1

paciente_vs_recurso.sort_values(by=['Tipo_de_paciente', 'Recurso'])

,Tipo_de_paciente,Recurso,Usa_recurso
0,TP1,R1,1
9,TP1,R10,1
1,TP1,R2,0
2,TP1,R3,0
3,TP1,R4,0
...,...,...,...
94,TP9,R5,0
95,TP9,R6,0
96,TP9,R7,1
97,TP9,R8,1


In [477]:
# tipo de paciente vs tempo de estadia
amostras_normais = np.random.normal(loc=estadia_max / 2, scale=estadia_max / 5, size=num_tipos_pacientes)
amostras_limite = np.clip(np.round(amostras_normais), 1, estadia_max)
amostras_limite = [max(elem, 0)  for elem in amostras_limite]
estadia = amostras_limite #np.random.randint(1, estadia_max, size=num_tipos_pacientes)

#print(tipo_paciente)
#print(estadia)

tipo_paciente_estadia_df = pd.DataFrame({'Tipo_de_paciente': tipo_paciente, 'Estadia': estadia})
display(tipo_paciente_estadia_df)

,Tipo_de_paciente,Estadia
0,TP1,2.0
1,TP2,1.0
2,TP3,1.0
3,TP4,3.0
4,TP5,2.0
5,TP6,2.0
6,TP7,3.0
7,TP8,3.0
8,TP9,2.0
9,TP10,3.0


In [478]:
# tipo de paciente vs área populacional vs dia de entrada vs qtd pacientes
area_pop = ["AP" + str(i) for i in range(1, num_areas_pop + 1)]
dias_planejados = [int(i) for i in range(1, horizonte_planejado + 1)] 
tipo_paciente = ["TP" + str(i) for i in range(1, num_tipos_pacientes + 1)]


qtd_pacientes_normais = np.random.normal(
    loc=qtd_pacientes_por_tipo_area_max / 2, 
    scale=qtd_pacientes_por_tipo_area_max/5, 
    size=num_tipos_pacientes * num_areas_pop * horizonte_planejado
)
qtd_pacientes_limite = np.clip(np.round(qtd_pacientes_normais), 0, qtd_pacientes_por_tipo_area_max)
qtd_pacientes_limite = [max(elem, 0) for elem in qtd_pacientes_limite]

qtd_pacientes = qtd_pacientes_limite #np.random.randint(0, qtd_pacientes_por_tipo_area_max, size=num_tipos_pacientes*num_areas_pop*horizonte_planejado)

tipo_paciente_df = np.tile(tipo_paciente, num_areas_pop*horizonte_planejado)

area_pop_df_in = np.tile(area_pop, num_tipos_pacientes*horizonte_planejado)
area_pop_df = sorted(area_pop_df_in)

dias_planejados_df_in1 = np.tile(dias_planejados, num_areas_pop)

dias_planejados_df_in2 = sorted(dias_planejados_df_in1)

dias_planejados_df = np.tile(dias_planejados_df_in2, num_tipos_pacientes)

area_pop_qtd_pac_df = pd.DataFrame({'Area_pop': area_pop_df, 'Tipo_de_paciente': tipo_paciente_df, 'Qtd_pacientes': qtd_pacientes, 'Dia': dias_planejados_df})
display(area_pop_qtd_pac_df)

,Area_pop,Tipo_de_paciente,Qtd_pacientes,Dia
0,AP1,TP1,128.0,1
1,AP1,TP2,186.0,1
2,AP1,TP3,285.0,1
3,AP1,TP4,229.0,1
4,AP1,TP5,188.0,1
...,...,...,...,...
2995,AP9,TP6,210.0,10
2996,AP9,TP7,227.0,10
2997,AP9,TP8,167.0,10
2998,AP9,TP9,159.0,10


In [479]:
# hospitais vs recursos vs quantidade de recursos
hospitais = ["H" + str(i) for i in range(1, num_hospitais + 1)]

media = (qtd_recursos_por_hospital_min + qtd_recursos_por_hospital_max) / 2
desvio_padrao = (qtd_recursos_por_hospital_max - qtd_recursos_por_hospital_min) / 6
qtd_recursos_normais = np.random.normal(
    loc=media, 
    scale=desvio_padrao, 
    size=num_recursos * num_hospitais
)
qtd_recursos_limite = np.clip(np.round(qtd_recursos_normais), qtd_recursos_por_hospital_min, qtd_recursos_por_hospital_max)
qtd_recursos_limite = [max(elem,0) for elem in qtd_recursos_limite]

qtd_recursos = qtd_recursos_limite # np.random.randint(qtd_recursos_por_hospital_min, qtd_recursos_por_hospital_max, size=num_recursos*num_hospitais)

hospitais_df = np.tile(hospitais, num_recursos)
recursos_df_in = np.tile(recursos, num_hospitais)
recursos_df = sorted(recursos_df_in)
#print(hospitais_df)
#print(qtd_recursos)

hospitais_vs_rec_qtd_df = pd.DataFrame ({'Hospital': hospitais_df, 'Recurso': recursos_df, 'Qtd_recurso': qtd_recursos})
display(hospitais_vs_rec_qtd_df.sort_values(by=['Hospital','Recurso']))

,Hospital,Recurso,Qtd_recurso
0,H1,R1,119.0
15,H1,R10,127.0
30,H1,R2,130.0
45,H1,R3,109.0
60,H1,R4,126.0
...,...,...,...
83,H9,R5,111.0
98,H9,R6,110.0
113,H9,R7,112.0
128,H9,R8,114.0


In [480]:
# Função para calcular a distância euclidiana
def calcular_distancia(p1, p2):
    return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

# Função principal
def calcular_distancias(hospitais, area_pop):
    # Dicionários para armazenar as localizações dos hospitais e áreas populacionais
    localizacao_hospitais = {}
    localizacao_area_pop = {}
    
    # Atribuir localizações aleatórias no plano cartesiano (x, y entre 0 e 100)
    for hospital in hospitais:
        localizacao_hospitais[hospital] = (random.randint(0, size_square), random.randint(0, size_square))
        
    for area in area_pop:
        localizacao_area_pop[area] = (random.randint(0, size_square), random.randint(0, size_square))
    
    # Criar lista para armazenar as distâncias
    distancias = []
    
    # Calcular as distâncias euclidianas entre cada hospital e cada área populacional
    for hospital in hospitais:
        for area in area_pop:
            dist = calcular_distancia(localizacao_hospitais[hospital], localizacao_area_pop[area])
            distancias.append([hospital, area, round(dist, 2)])  # Arredonda a distância para 2 casas decimais
    
    # Criar DataFrame
    df = pd.DataFrame(distancias, columns=["Hospital", "Area_pop", "Distancia"])
    
    return df


In [481]:
# hospitais vs area pop vs distância (gerando aleatoriamente pontos num plano cartesiano)
hospitais_vs_area_pop_dist_df = calcular_distancias(hospitais, area_pop)
hospitais_vs_area_pop_dist_df

,Hospital,Area_pop,Distancia
0,H1,AP1,66.71
1,H1,AP2,68.36
2,H1,AP3,41.01
3,H1,AP4,81.40
4,H1,AP5,46.69
...,...,...,...
445,H15,AP26,12.21
446,H15,AP27,27.17
447,H15,AP28,84.90
448,H15,AP29,45.34


In [482]:
# # hospitais vs area pop vs distância
# distancia = np.random.randint(1, distancia_max, size=num_hospitais*num_areas_pop)

# hospitais_dist_df = np.tile(hospitais, num_areas_pop)
# area_pop_dist_df_in = np.tile(area_pop, num_hospitais)
# area_pop_dist_df = sorted(area_pop_dist_df_in)

# hospitais_vs_area_pop_dist_df = pd.DataFrame ({'Hospital': hospitais_dist_df, 'Area_pop': area_pop_dist_df, 'Distancia': distancia})
# display(hospitais_vs_area_pop_dist_df.sort_values(by=['Hospital','Area_pop']))

In [483]:
area_pop

['AP1',
 'AP2',
 'AP3',
 'AP4',
 'AP5',
 'AP6',
 'AP7',
 'AP8',
 'AP9',
 'AP10',
 'AP11',
 'AP12',
 'AP13',
 'AP14',
 'AP15',
 'AP16',
 'AP17',
 'AP18',
 'AP19',
 'AP20',
 'AP21',
 'AP22',
 'AP23',
 'AP24',
 'AP25',
 'AP26',
 'AP27',
 'AP28',
 'AP29',
 'AP30']

In [484]:
# hospitais vs area pop vs distância
num_areas_pop

30

In [485]:
# condição inicial: hospitais vs tipo paciente vs qtd vs data de saída
data_saida = np.random.randint(1, data_saida_pac_init_max, size=num_hospitais*num_tipos_pacientes)
qtd_pacientes_liberados_init = np.random.randint (0, qtd_pacientes_liberados_init_max, size = num_hospitais*num_tipos_pacientes)

print(hospitais)
print(tipo_paciente)
print(qtd_pacientes_liberados_init)
print(data_saida)

hospitais_lib_df_in = np.tile(hospitais, num_tipos_pacientes)
hospitais_lib_df = sorted(hospitais_lib_df_in)
tipo_paciente_lib_df = np.tile(tipo_paciente, num_hospitais)

cond_init_df = pd.DataFrame({'Hospital': hospitais_lib_df, 'Tipo_de_paciente': tipo_paciente_lib_df, 'Qtd_pacientes_liberados': qtd_pacientes_liberados_init, 'Dia': data_saida})
display(cond_init_df)

['H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9', 'H10', 'H11', 'H12', 'H13', 'H14', 'H15']
['TP1', 'TP2', 'TP3', 'TP4', 'TP5', 'TP6', 'TP7', 'TP8', 'TP9', 'TP10']
[ 1 18  1 31 61 68 57 40 24 35 69 22 38 44 57  5  8 20 66 22 52 11 17 67
 19 45 54 22  2 45 48 67 44 15 69 62 10 69 47 35 39 21 57 63 43  1 65 42
 45 18 52 29 46 13 36  1 18 10 38 64 25 31 17 21 32 51 61  7 36 41 23 33
 37 16  2  8 57 68 65  5 21 21  8 58 24 42 51 61 61 43 41 65 64 42 45 55
 44 61 42 17 37 62 36 57 56 67 10 25 65 23 17 28 63 34 17 52 45 12 45 56
 21 14 23 35 47 17 29 35 64 52 29  2 18 56 38  9 65 42  9 60 15 20 26  6
  3 62 52  4  8 47]
[2 3 2 3 2 3 2 2 1 2 3 1 2 3 2 3 3 2 2 3 3 1 1 2 2 3 1 2 1 2 3 2 2 1 3 3 3
 3 2 1 2 1 2 1 2 1 2 1 3 1 3 1 1 3 2 2 1 2 2 2 3 3 3 2 1 1 1 3 1 3 2 1 3 1
 2 2 3 2 1 2 1 1 3 1 2 2 1 1 3 3 2 3 2 1 1 3 3 1 1 2 3 3 1 1 2 2 1 1 3 2 2
 2 3 2 3 2 1 3 2 2 3 2 1 2 3 1 1 2 3 1 3 3 3 1 3 1 3 3 1 2 3 3 1 2 2 1 3 3
 3 2]


,Hospital,Tipo_de_paciente,Qtd_pacientes_liberados,Dia
0,H1,TP1,1,2
1,H1,TP2,18,3
2,H1,TP3,1,2
3,H1,TP4,31,3
4,H1,TP5,61,2
...,...,...,...,...
145,H9,TP6,62,1
146,H9,TP7,52,3
147,H9,TP8,4,3
148,H9,TP9,8,3


In [486]:
paciente_vs_recurso.to_csv('../data/paciente_vs_recurso.csv', index=False)
tipo_paciente_estadia_df.to_csv('../data/tipo_paciente_estadia.csv', index=False)
area_pop_qtd_pac_df.to_csv('../data/area_pop_qtd_pacientes.csv', index=False)
hospitais_vs_rec_qtd_df.to_csv('../data/hospitais_vs_recursos_qtd.csv', index=False)
hospitais_vs_area_pop_dist_df.to_csv('../data/hospitais_vs_area_pop_dist.csv', index=False)
cond_init_df.to_csv('../data/cond_init.csv', index=False)